In [1]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import transformers
import torch
import os
import nltk
import pandas as pd
import torch
import numpy as np
from jinja2 import Template
import pickle
from collections import defaultdict
import copy

from transformers.generation.logits_process import LogitsProcessorList
logits_processor = LogitsProcessorList()

import sys
sys.path.append('/scratch/users/bozyurt20/hpc_run/utilities')
sys.path.append("/scratch/users/bozyurt20/hpc_run/blobs/")
from util_research import *

max_len = 512
num_layers = 24
d_model = 4096

tokenizer = AutoTokenizer.from_pretrained("bigscience/T0pp", truncation_side="right", add_prefix_space=True)
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

prng = np.random.RandomState(1)

In [3]:
model = AutoModelForSeq2SeqLM.from_pretrained("bigscience/T0pp", device_map='balanced')
#model.eval()

In [ ]:
#model.parallelize()

In [47]:
long_contexts = [

"""John went to London. London was amazing and it was very crowded.

London is the capital and largest city of England and the United Kingdom, with a population of just under 9 million. 
It stands on the River Thames in south-east England at the head of a 50-mile (80 km) estuary down to the North Sea, and has been 
a major settlement for two millennia. The City of London, its ancient core and financial centre, was founded by the Romans as Londinium
and retains its medieval boundaries.
The City of Westminster, to the west of the City of London, has for centuries hosted the national government and parliament. 
Since the 19th century, the name "London" has also referred to the metropolis around this core, historically split between 
the counties of Middlesex, Essex, Surrey, Kent, and Hertfordshire, which since 1965 has largely comprised Greater London, 
which is governed by 33 local authorities and the Greater London Authority.

John was very happy. He was having a great time there. Where is John?""",

"""John went to London. John has a special name.

John is a common male given name in the English language ultimately of Hebrew origin. 
The English form is from Middle English Ion, Ihon, Jon, Jan (mid-12c.), itself from Old French Jan, Jean, Jehan (Modern French Jean), from Medieval Latin Johannes, altered form of Late Latin Ioannes,
or the Middle English personal name is directly from Medieval Latin, which is from the Greek name Ioannis, originally borne by Hellenized Jews transliterating the Hebrew name Yochanan, the contracted form of the longer name Yehochanan, meaning "Yahweh is Gracious" or "Yahweh is Merciful". 
There are numerous forms of the name in different languages; these were formerly often simply translated as "John" in English, but are increasingly left in their native forms.

John was very happy. He was having a great time there. Where is John?""",
    
"""John went to London. Mary had a toothache. 

Toothache, also known as dental pain, is pain in the teeth or their supporting structures, caused by dental diseases or pain referred to the teeth by non-dental diseases.
When severe it may impact sleep, eating, and other daily activities.

John was very happy. He was having a great time there. Where is John?""",
    
]


In [54]:
long_contexts[2]

'John went to London. Mary had a toothache. \n\nToothache, also known as dental pain, is pain in the teeth or their supporting structures, caused by dental diseases or pain referred to the teeth by non-dental diseases.\nWhen severe it may impact sleep, eating, and other daily activities.\n\nJohn was very happy. He was having a great time there. Where is John?'

In [56]:

results = {}
for num_context, long_context in enumerate(long_contexts):
    
    input_ids = tokenizer.encode(long_context, return_tensors="pt")
    input_ids = input_ids.to(device)
    with torch.no_grad():
        encoder_out = model.encoder(input_ids)

    nu = 3 * encoder_out.last_hidden_state.std().item()

    answer = "London"

    decoder_input_ids = tokenizer.encode('<pad>', add_special_tokens=False, return_tensors="pt").cuda()
    encoder_last_hiddens = encoder_out.last_hidden_state
    B, T, d = encoder_last_hiddens.shape

    encoder_out_copy = copy.deepcopy(encoder_out)
    with torch.no_grad():
        model_out = model.generate(decoder_input_ids=decoder_input_ids, 
                                   encoder_outputs=encoder_out_copy,
                                   return_dict_in_generate=True,
                                   max_new_tokens=1,
                                   output_scores=True)
    next_token_scores_clean = torch.nn.functional.softmax(model_out.scores[0].float(), dim=-1)  # (batch_size * num_beams, vocab_size)
    probability_clean = next_token_scores_clean[0][tokenizer.encode(answer)[0]].item()

    noisy_probabilities = []

    for i in range(T):
        prob_corrects = []
        
        for _ in range(50):
            encoder_out_copy = copy.deepcopy(encoder_out)
            encoder_out_copy.last_hidden_state[0, i, :] += nu * torch.from_numpy(prng.randn(d)).cuda()

            with torch.no_grad():
                model_out = model.generate(decoder_input_ids=decoder_input_ids, 
                                       encoder_outputs=encoder_out_copy,
                                       return_dict_in_generate=True,
                                       max_new_tokens=1,
                                       output_scores=True)

            next_token_scores = torch.nn.functional.softmax(model_out.scores[0].float(), dim=-1)  # (batch_size * num_beams, vocab_size)
            probability_correct = next_token_scores[0][tokenizer.encode(answer)[0]].item()
            prob_corrects.append(probability_correct)
        
        probability_correct = sum(prob_corrects) / len(prob_corrects)

        noisy_probabilities.append(probability_correct)
        
    results[num_context] = [long_context, probability_clean, noisy_probabilities]

new code working-modeling_t5
new code working-utils
llama generation happening.
new code working-modeling_t5
new code working-utils
llama generation happening.
new code working-modeling_t5
new code working-utils
llama generation happening.
new code working-modeling_t5
new code working-utils
llama generation happening.
new code working-modeling_t5
new code working-utils
llama generation happening.
new code working-modeling_t5
new code working-utils
llama generation happening.
new code working-modeling_t5
new code working-utils
llama generation happening.
new code working-modeling_t5
new code working-utils
llama generation happening.
new code working-modeling_t5
new code working-utils
llama generation happening.
new code working-modeling_t5
new code working-utils
llama generation happening.
new code working-modeling_t5
new code working-utils
llama generation happening.
new code working-modeling_t5
new code working-utils
llama generation happening.
new code working-modeling_t5
new code wo

In [57]:
results.keys()

dict_keys([0, 1, 2])

In [66]:
print(results[2][1])

0.8617010712623596


In [64]:
for token in tokenizer.tokenize(results[2][0]) + ["eos"]:
    print(token)

▁John
▁went
▁to
▁London
.
▁Mary
▁had
▁
a
▁tooth
ache
.
▁Too
th
ache
,
▁also
▁known
▁as
▁dental
▁pain
,
▁is
▁pain
▁in
▁the
▁teeth
▁or
▁their
▁supporting
▁structures
,
▁caused
▁by
▁dental
▁diseases
▁or
▁pain
▁
referred
▁to
▁the
▁teeth
▁by
▁non
-
den
tal
▁diseases
.
▁When
▁severe
▁it
▁may
▁impact
▁sleep
,
▁eating
,
▁and
▁other
▁daily
▁activities
.
▁John
▁was
▁very
▁happy
.
▁He
▁was
▁having
▁
a
▁great
▁time
▁there
.
▁Where
▁is
▁John
?
eos


In [65]:
for noise in results[2][2]:
    print(results[2][1] - noise)

0.1107579109072685
0.23481345331296322
0.2772637069690973
0.483073193076998
0.12470616988837724
0.15644075603457164
0.08169602669775489
0.17373317292891444
0.15368119318969553
0.10758988297311589
0.07833860181272034
0.04178702150471514
0.05818072199821467
0.02097865819931033
0.04514322817325589
0.011356370449066122
0.02169199943542477
0.011652671098709155
0.03069759845733644
0.013573898077011148
0.028962724581360844
0.0780500395782292
0.03290777623653407
0.05128032684326167
0.03947939859368488
0.08236993331305709
0.03315562129020688
0.015395601987838736
0.6068584417703141
0.01204342007637027
0.051998310427879946
0.11012413591146464
0.05703794704750176
0.020239337086677578
0.007618889808654772
0.025656441450119027
0.036611802121624404
0.01580418527126315
0.14478612303268168
0.010572584867477364
0.06777481570839883
0.07331064639613027
0.005160518884658849
0.02586354255676271
0.013316227197647135
0.054853156208992004
0.09709212754387408
0.009124014377594025
0.01143781065940852
0.046961533

In [22]:
for noise in noisy_probabilities:
    print(probability_clean - noise)

0.3601031005382538
0.49290651082992554
0.3648990988731384
0.5019066035747528
0.7402898296713829
0.27560412883758545
0.06724566221237183
0.1525387167930603
0.7881833402116172
0.4345082938671112
0.7587933745235205
0.10451668500900269
0.31403735280036926
0.26824915409088135
0.12643420696258545
0.043530046939849854
0.045642852783203125
-0.016503512859344482
-0.000763237476348877
0.5448595881462097
0.3146081268787384
-0.03180968761444092
0.23266011476516724
0.04303687810897827
0.3586311936378479
0.02249300479888916
0.04395049810409546
-0.05731189250946045
0.04973256587982178
-0.028364062309265137
-0.006671607494354248
0.6202106326818466
0.003538966178894043
0.025958001613616943
0.018369972705841064
0.0060204267501831055
0.05739659070968628
-0.024379253387451172
0.05842775106430054
-0.008187830448150635
0.02723979949951172
0.0023075342178344727
-0.012070298194885254
0.036861300468444824
0.0020421743392944336
0.032908082008361816
-0.02676522731781006
0.1535056233406067
0.08515757322311401
0.0

In [14]:
tokens = tokenizer.tokenize(long_context) + ["eos"]

for token in tokens:
    print(token)

▁John
▁went
▁to
▁London
.
▁John
▁has
▁
a
▁special
▁name
.
▁John
▁is
▁
a
▁common
▁male
▁given
▁name
▁in
▁the
▁English
▁language
▁ultimately
▁of
▁Hebrew
▁origin
.
▁The
▁English
▁form
▁is
▁from
▁Middle
▁English
▁I
on
,
▁I
h
on
,
▁Jon
,
▁Jan
▁(
mid
-12
c
.
),
▁itself
▁from
▁Old
▁French
▁Jan
,
▁Jean
,
▁Je
han
▁(
Mod
er
n
▁French
▁Jean
),
▁from
▁Medi
e
val
▁Latin
▁Johannes
,
▁altered
▁form
▁of
▁Late
▁Latin
▁Ioan
nes
,
▁or
▁the
▁Middle
▁English
▁personal
▁name
▁is
▁directly
▁from
▁Medi
e
val
▁Latin
,
▁which
▁is
▁from
▁the
▁Greek
▁name
▁Ioan
n
is
,
▁originally
▁
borne
▁by
▁Hell
en
ized
▁Jews
▁trans
liter
ating
▁the
▁Hebrew
▁name
▁Yo
chan
an
,
▁the
▁contracted
▁form
▁of
▁the
▁longer
▁name
▁Ye
hoch
an
an
,
▁meaning
▁"
Y
a
h
we
h
▁is
▁Gr
acious
"
▁or
▁"
Y
a
h
we
h
▁is
▁Merci
ful
".
▁There
▁are
▁numerous
▁forms
▁of
▁the
▁name
▁in
▁different
▁languages
;
▁these
▁were
▁
formerly
▁often
▁simply
▁translated
▁as
▁"
John
"
▁in
▁English
,
▁but
▁are
▁increasingly
▁left
▁in
▁their
▁native
▁forms
.
▁John
▁w

In [15]:
len(tokens)

212

In [16]:
len(noisy_probabilities)

212

In [18]:
len(tokenizer.tokenize(long_context))

211

In [51]:
with open("/kuacc/users/bozyurt20/hpc_run/predictions/adidng_noise_long_context.txt", "wb") as f:
    pickle.dump(results, f)

In [ ]:
tokens = tokenizer.tokenize(long_context) + ["eos"]
result_dic = defaultdict(list)

for j, prob in enumerate(noisy_probabilities):
    print(tokens[j], probability_clean - prob)
    result_dic[str(j)].append(answer_probability - prob)
    
    print()
    print()
    print()

In [ ]:
model_outs = []
decoder_input_ids = tokenizer.encode('<pad>', add_special_tokens=False, return_tensors="pt").cuda()

my_dic = {
    "prompt": [],
    "answer": [],
    "answer probability": [],
    "noisy probabilities": []
}

for data_point in data_points_one_sentence:
    prompt = data_point.prompt_1
    answer = data_point.answer_1
    
    input_ids = tokenizer.encode(prompt, return_tensors="pt").to(model.encoder.device)
    
    with torch.no_grad():
        encoder_out = model.encoder(input_ids)
    
    encoder_last_hiddens = encoder_out.last_hidden_state
    B, T, d = encoder_last_hiddens.shape
    
    assert T == 229
    
    encoder_out_copy = copy.deepcopy(encoder_out)
    with torch.no_grad():
        model_out = model.generate(decoder_input_ids=decoder_input_ids, 
                                   encoder_outputs=encoder_out_copy,
                                   return_dict_in_generate=True,
                                   max_new_tokens=1,
                                   output_scores=True)
    next_token_scores = torch.nn.functional.softmax(model_out.scores[0].float(), dim=-1)  # (batch_size * num_beams, vocab_size)
    probability_correct = next_token_scores[0][tokenizer.encode(answer)[0]].item()    
    
    my_dic["prompt"].append(prompt)
    my_dic["answer"].append(answer)
    my_dic["answer probability"].append(probability_correct)
    
    noisy_probabilities = []
    
    for i in range(T):
        encoder_out_copy = copy.deepcopy(encoder_out)
        encoder_out_copy.last_hidden_state[0, i, :] += nu * torch.from_numpy(prng.randn(d)).cuda()

        model_out = model.generate(decoder_input_ids=decoder_input_ids, 
                               encoder_outputs=encoder_out_copy,
                               return_dict_in_generate=True,
                               max_new_tokens=1,
                               output_scores=True)
        
        next_token_scores = torch.nn.functional.softmax(model_out.scores[0].float(), dim=-1)  # (batch_size * num_beams, vocab_size)
        probability_correct = next_token_scores[0][tokenizer.encode(answer)[0]].item()
        
        noisy_probabilities.append(probability_correct)
    
    my_dic["noisy probabilities"].append(noisy_probabilities)